In [2]:
import pandas as pd
import glob
import os
import geopandas 
import geopy 
from geopy.geocoders import Nominatim


In [6]:
data = pd.concat([pd.read_excel(f, 'Results') for f in glob.glob('./data/*.xlsx')], ignore_index = True)

In [6]:
data = pd.read_pickle('./data/data_almost_complete.pkl')

In [8]:
london_postcodes = pd.read_csv('./london_postcodes-ons-postcodes-directory-feb22.csv')

In [9]:
def data_processing(data, london_data): 
    data = data.loc[(data['Inactive'] == 'No')]
    data = data[data['Primary UK SIC (2007) code'].notna()]
    data = data[data['Primary trading address'].notna()]
    data = data.drop_duplicates(keep=False)
    data['postcode'] = data['Primary trading address'].apply(lambda x: x.split(',')[-1])
    data['postcode'] = data['postcode'].apply(lambda x: x[1:])
    london_postcodes = london_data['pcd'].tolist()
    london_df = data[data['postcode'].isin(london_postcodes)]
    london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')
    return london_df 

In [10]:
london_df = data_processing(data , london_postcodes)

In [14]:
london_intangibles = london_df[london_df['Primary UK SIC (2007) code'].str.contains(
    "^85|^92|^96|^59|^60|^61|^62|^63|^64|^66|^69|^70|^71|^72|^73|^74|^79|^82|^77|^78")]

In [17]:
london_intangibles

,Unnamed: 0,Company name,Inactive,Quoted,Branch,OwnData,Woco,R/O Full Postcode,Primary UK SIC (2007) code,Primary trading address,Website,R/O Address,"R/O Address, line 1","R/O Address, line 2","R/O Address, line 3",R/O Full Postcode.1,Primary trading address City,postcode
83,84.0,MOORE STEPHENS INTERNATIONAL LIMITED,No,No,No,No,No,SE1 3TQ,96090.0,"5 Market Yard Mews, 194-204 Bermondsey Street,...",www.sequel.com,"5 Market Yard Mews, 194-204 Bermondsey Street,...",5 Market Yard Mews,194-204 Bermondsey Street,NaN,SE1 3TQ,London,SE1 3TQ
145,146.0,MORGAN STANLEY GROUP (EUROPE),No,No,No,No,No,E14 4AD,70100.0,"20 Bank Street, Canary Wharf, London, London, ...",www.morganstanley.com,"20 Bank Street, Canary Wharf, London, E14 4AD",20 Bank Street,Canary Wharf,NaN,E14 4AD,London,E14 4AD
205,206.0,CHURCHVILLE LIMITED,No,No,No,No,No,CR0 0YN,78200.0,"3rd Floor Suffolk House, George Street, Croydo...",www.theoaklandpartnership.com,"3rd Floor Suffolk House, George Street, Croydo...",3rd Floor Suffolk House,George Street,NaN,CR0 0YN,Croydon,CR0 0YN
256,257.0,INTU 2027 LIMITED,No,No,No,No,No,W1K 3JZ,64209.0,"c/o Apam Ltd, 4th Floor, 84, Grosvenor Street,...",www.capital.shopping-centres.co.uk,"c/o Apam Ltd, 4th Floor, 84, Grosvenor Street,...","c/o Apam Ltd, 4th Floor, 84",Grosvenor Street,NaN,W1K 3JZ,London,W1K 3JZ
304,305.0,MICROEYE LIMITED,No,No,No,No,No,DA1 4SU,82990.0,"91 Shearwood Crescent, Thames Road, Dartford, ...",www.microeyeservices.com,"91 Shearwood Crescent, Thames Road, Dartford, ...",91 Shearwood Crescent,Thames Road,NaN,DA1 4SU,Dartford,DA1 4SU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766918,74820.0,EXCELLENT MEDIA LTD,No,No,No,No,No,W1W 7LT,63120.0,"85 Great Portland Street, London, London, W1W 7LT",www.excellentmedia.co.uk,"85 Great Portland Street, London, W1W 7LT",85 Great Portland Street,NaN,NaN,W1W 7LT,London,W1W 7LT
2766963,74865.0,PROFIT TRADING LTD,No,No,No,No,No,SW6 7BS,82990.0,"21 Letterstone Road, Fulham, London, London, S...",profittrading.net,"21 Letterstone Road, Fulham, London, SW6 7BS",21 Letterstone Road,Fulham,NaN,SW6 7BS,London,SW6 7BS
2766971,74873.0,TOTALPLUS SERVICES LTD.,No,No,No,No,No,W1B 5TE,78109.0,"c/o Gpc Financial Management, 423 Linen Hall, ...",www.totalpluscleaners.com,"c/o Gpc Financial Management, 423 Linen Hall, ...",c/o Gpc Financial Management,423 Linen Hall,NaN,W1B 5TE,London,W1B 5TE
2767055,74957.0,GOLDSTAR DRY CLEANERS (UK) LIMITED,No,No,No,No,No,E14 7DL,96010.0,"330 Burdett Road, London, London, E14 7DL",www.goldstardrycleaning.co.uk,"330 Burdett Road, London, E14 7DL",330 Burdett Road,NaN,NaN,E14 7DL,London,E14 7DL


In [135]:
def coordinates_finder(address):
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(address)
    try:
        coordinates = '{}, {}'.format(location.latitude, location.longitude)
        print(coordinates)
    except AttributeError:
        coordinates = 'not found'
    return coordinates

In [ ]:
london_df['coordinates'] = london_df['Primary trading address'].apply(lambda x: coordinates_finder(x))

https://data.london.gov.uk/dataset/postcode-directory-for-london